In [1]:
%env WANDB_NOTEBOOK_NAME Train.ipynb
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 1
%env PYTORCH_CUDA_ALLOC_CONF backend:cudaMallocAsync

env: WANDB_NOTEBOOK_NAME=Train.ipynb
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
env: PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync


In [2]:
from mp_20_utils import load_all_data
from tokenization import MASK_SITE
device = 'cuda'
datasets_pd, torch_datasets, site_to_ids, element_to_ids, spacegroup_to_ids, max_len = load_all_data(
    device=device, allow_retokenize=False)

In [3]:
from itertools import chain
from operator import itemgetter
max_enumeration = max(map(
    lambda dataframe: max(chain.from_iterable(dataframe.symmetry_sites_enumeration_padded)),
    datasets_pd.values()))
print(max_enumeration)

7


In [4]:
import torch
# UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
torch.set_float32_matmul_precision('high')
from wyckoff_transformer import WyckoffTransformerModel, WyckoffTrainer
d_hid = 170  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 3  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 1  # number of heads in ``nn.MultiheadAttention``
dropout = 0.1  # dropout probability
n_space_groups = len(spacegroup_to_ids)
# Not all 230 space groups are present in the data
model = WyckoffTransformerModel(
    n_space_groups = n_space_groups,
    n_sites = len(site_to_ids),
    n_elements = len(element_to_ids),
    max_enumeration = max_enumeration,
    d_space_groups = 16,
    d_sites = 64,
    d_species = 64,
    nhead=nhead,
    d_hid=d_hid,
    max_len=max_len,
    nlayers=nlayers,
    dropout=dropout).to(device)
#model = torch.compile(model)

In [5]:
import torch
trainer = WyckoffTrainer(model, torch_datasets, max_len,
                         site_mask=torch.tensor(site_to_ids[MASK_SITE]).to(device),
                         element_pad=torch.tensor(element_to_ids["PAD"]).to(device),
                         site_pad=torch.tensor(site_to_ids["PAD"]).to(device))
trainer.train(epochs=4000, val_period=10)

wandb: Currently logged in as: kazeev. Use `wandb login --relogin` to force relogin


Traceback (most recent call last):
  File "/home/kna/WyckoffTransformer_dev/wyckoff_transformer.py", line 265, in train
    self.train_step()
  File "/home/kna/WyckoffTransformer_dev/wyckoff_transformer.py", line 223, in train_step
    loss_element, loss_site, loss_enumeration = self.get_token_loss(self.torch_datasets["train"], known_seq_len)
  File "/home/kna/WyckoffTransformer_dev/wyckoff_transformer.py", line 208, in get_token_loss
    output_enumeration = self.model(dataset["spacegroup_number"], **data_enumeration,
  File "/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-psWxdoGj-py3.10/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1511, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/home/kna/.cache/pypoetry/virtualenvs/wyckofftransformer-psWxdoGj-py3.10/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1520, in _call_impl
    return forward_call(*args, **kwargs)
  File "/home/kna/WyckoffTransformer_dev/wyckoff_trans

known_seq_len,▂▁█▃
lr,▁▁▁▁
train_loss_batch,▅█▁▂
train_loss_element,▆█▁▂
train_loss_enumeration,█▇▁▂
train_loss_site,▄█▁▂
known_seq_len,8
lr,5.0
train_loss_batch,19503.625
train_loss_element,10289.24805
train_loss_enumeration,2938.37598


OutOfMemoryError: Allocation on device 0 would exceed allowed memory. (out of memory)
Currently allocated     : 42.35 GiB
Requested               : 900.59 MiB
Device limit            : 44.52 GiB
Free (according to CUDA): 8.94 MiB
PyTorch limit (set by user-supplied memory fraction)
                        : 17179869184.00 GiB